In [2]:
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

train_data = np.load('MLProject2024/fashion_train.npy')
test_data = np.load('MLProject2024/fashion_test.npy')

train_images = train_data[:, :-1] 
train_labels = train_data[:, -1].astype(int)
test_images = test_data[:, :-1]
test_labels = test_data[:, -1].astype(int)

mean = np.mean(train_images, axis=0)
std = np.std(train_images, axis=0)
train_images = (train_images - mean) / std
test_images = (test_images - mean) / std 

#pca
pca = PCA(n_components=100)
train_images = pca.fit_transform(train_images)
test_images= pca.transform(test_images)

In [ ]:

from sklearn.neural_network import MLPClassifier

nn=MLPClassifier(hidden_layer_sizes=(200,200), activation='relu',  learning_rate='adaptive', learning_rate_init=0.01, max_iter=200)
nn.fit(train_images, train_labels)
print(nn.score(train_images, train_labels))
print(nn.score(test_images, test_labels))


#best combination of parameters



0.9837
0.838


In [11]:
import optuna
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function for optimization
def objective(trial):
    # Suggest hyperparameters
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [(64, 32), (128, 64, 32), (256, 128)])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    alpha = trial.suggest_loguniform('alpha', 1e-5, 1e-1)
    learning_rate_init = trial.suggest_loguniform('learning_rate_init', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64])
    
    # Create the model
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, alpha=alpha,
                           learning_rate_init=learning_rate_init, batch_size=batch_size, max_iter=200)
    
    # Evaluate the model using cross-validation
    scores = cross_val_score(model, train_images, train_labels, cv=3, scoring='accuracy')
    return scores.mean()

# Perform optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Best parameters
print("Best parameters found: ", study.best_params)


c:\Programs\Anaconda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-01-01 17:16:41,172] A new study created in memory with name: no-name-20b099db-fdf8-4f36-8ef4-6dfffda8c52e
c:\Programs\Anaconda\Lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (64, 32) which is of type tuple.
  warnings.warn(message)
c:\Programs\Anaconda\Lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (128, 64, 32) which is of type tuple.
  warnings.warn(message)
c:\Programs\Anaconda\Lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorica

Best parameters found:  {'hidden_layer_sizes': (256, 128), 'activation': 'relu', 'alpha': 0.006418956807683995, 'learning_rate_init': 0.0002899559538770737, 'batch_size': 32}


In [12]:
# Train the model using the best parameters Best parameters found:  {'hidden_layer_sizes': (256, 128), 'activation': 'relu', 'alpha': 0.006418956807683995, 'learning_rate_init': 0.0002899559538770737, 'batch_size': 32}
best_params = study.best_params
model = MLPClassifier(hidden_layer_sizes=best_params['hidden_layer_sizes'], activation=best_params['activation'],
                      alpha=best_params['alpha'], learning_rate_init=best_params['learning_rate_init'],
                      batch_size=best_params['batch_size'], max_iter=200)
model.fit(train_images, train_labels)

# Evaluate the model
train_score = model.score(train_images, train_labels)
test_score = model.score(test_images, test_labels)
print("Train accuracy: ", train_score)
print("Test accuracy: ", test_score)

Train accuracy:  1.0
Test accuracy:  0.855


c:\Programs\Anaconda\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
